# AIPI 510: Team Assignment 2 - Data Storage and Access (SQL)

## Owais Kamdar, Rajiv Raman

We are submitting our code for this assignment in the Python notebook format. **To run the code, click "Run All" to evaluate all the cells.** Comments throughout the code will indicate where the CREATE, READ, UPDATE, and DELETE requirements are satisfied.

Note: make sure that the filename string is the correct pathway to the data file.

In [168]:
import sqlite3
import pandas as pd
import os
from IPython.display import display

filename = "/Users/rajivraman/Downloads/tips.csv"; # change to match the pathway of tips.csv

# define the sqlquery function to execute a given SQL query on a given SQL database connection

def sqlquery(connection,prompt):
    try:
        cur = connection.cursor() # try to create a cursor for an SQL query
        cur.execute(prompt) # prompt parameter is fed in as SQL query
        results = cur.fetchall() # store all results from the query in a variable
        if len(results) > 0: # if the query returned a nonzero number of results
            return results # return the results
        else:
            return "No results found." # return the message that no data entry matched the query
        
    except sqlite3.Error as e:
        print(f"SQL query execution error: {e}") # print error message if the SQL query cannot be performed

try: 
    conn = sqlite3.connect("test1") # open the connection to an SQL database named test1
    print("Connected to the SQL Database! Executing all queries...\n")
    
    data = pd.read_csv(filename) # store the data from tips.csv in this variable
    
    try:
        data.to_sql('table1',conn,index=False,if_exists='replace') # the CREATE step - forms the SQL database from tips.csv
    except Exception as e:
        print(f"Data upload error: {e}") # print error message if data cannot be uploaded to the SQL database
        
    # the READ step is fully satisfied below - all 10 requested SQL queries plus 5 custom queries have been written
        
    # SQL query for #1. Retrieve the average tip percentage for each day of the week.
        
    sql1 = sqlquery(conn,"SELECT day, AVG(tip/total_bill * 100) FROM table1 GROUP BY day")
    
    # SQL query for #2. Find the minimum and maximum total bill amounts.
    
    sql2 = sqlquery(conn,"SELECT MIN(total_bill), MAX(total_bill) FROM table1")
    
    # SQL query for #3. Count the number of parties for each size.
    
    sql3 = sqlquery(conn,"SELECT size, COUNT(total_bill) FROM table1 GROUP BY size")
    
    # SQL query for #4. Retrieve the total bill and tip for parties of size 4 or more, where the tip percentage is greater than 15%.
    
    sql4 = sqlquery(conn,"SELECT size, total_bill, tip, tip/total_bill * 100 FROM table1 WHERE size >= 4 AND tip/total_bill * 100 > 15")
    
    # SQL query for #5. Retrieve the total bill, tip amount, and tip percentage for each combination of day and time, sorted by tip percentage in descending order.
    
    sql5 = sqlquery(conn,"SELECT day, time, tip, total_bill, tip/total_bill * 100 FROM table1 ORDER BY day, time, tip/total_bill * 100 DESC")
    
    # SQL query for #6. Find the average tip percentage for each combination of day, time, and smoker status.
    
    sql6 = sqlquery(conn,"SELECT day, time, smoker, AVG(tip/total_bill * 100) FROM table1 GROUP BY day, time, smoker")

    # SQL query for #7. Retrieve the total bill, tip amount, and tip percentage for each sex, sorted by total bill in descending order, and limit the results to the top 5 records.
    
    sql7 = sqlquery(conn,"SELECT sex, total_bill, tip, tip/total_bill * 100 FROM table1 ORDER BY total_bill DESC LIMIT 5")
    
    # SQL query for #8. Find the maximum and minimum tip percentage for each day and time combination, along with the corresponding total bill and tip amount.
    
    sql8 = sqlquery(conn,"SELECT day, time, tip/total_bill * 100, tip, total_bill FROM table1 WHERE tip/total_bill * 100 = (SELECT MAX(tip/total_bill * 100) FROM table1 AS temp WHERE temp.day = table1.day AND temp.time = table1.time) OR tip/total_bill * 100 = (SELECT MIN(tip/total_bill * 100) FROM table1 AS temp WHERE temp.day = table1.day AND temp.time = table1.time) ORDER BY day, time, tip/total_bill * 100")
    
    # SQL query for #9. Retrieve the total bill, tip amount, and tip percentage for parties of size 4 or more, where the tip percentage is greater than 15%, and the total bill is between $50 and $100.
    
    sql9 = sqlquery(conn,"SELECT total_bill, tip, tip/total_bill * 100 FROM table1 WHERE size >= 4 AND tip/total_bill * 100 > 15 AND total_bill <= 100 AND total_bill >= 50")
    
    # SQL query for #10. Find the average tip percentage for each combination of day, time, and smoker status, but only include combinations with more than 5 records.
    
    sql10 = sqlquery(conn,"SELECT day, time, smoker, AVG(tip/total_bill * 100) FROM table1 GROUP BY day, time, smoker HAVING COUNT(*) > 5")
    
    # Extra SQL query #1. Compute the total tip per day.
    
    xsql1 = sqlquery(conn,"SELECT day, SUM(tip) AS total_tip FROM table1 GROUP BY day ORDER BY total_tip DESC")
    
    # Extra SQL query #2. Compute the highest total bill for each party size.
    
    xsql2 = sqlquery(conn,"SELECT size, MAX(total_bill) AS max_total_bill FROM table1 GROUP BY size ORDER BY size")
    
    # Extra SQL query #3. Compute the average tip for each gender.
    
    xsql3 = sqlquery(conn,"SELECT sex, AVG(tip) AS avg_tip FROM table1 GROUP BY sex")
    
    # Extra SQL query #4. Compute the top 3 highest tips for each day.
    
    xsql4 = sqlquery(conn,"WITH ranked_tips AS (SELECT day, tip, ROW_NUMBER() OVER (PARTITION BY day ORDER BY tip DESC) AS rank FROM table1) SELECT day, tip FROM ranked_tips WHERE rank <= 3 ORDER BY day, tip DESC")
    
    # Extra SQL query #5. Compute the number of smokers vs. number of non-smokers by time.
    
    xsql5 = sqlquery(conn, "SELECT time, smoker, COUNT(*) AS count FROM table1 GROUP BY time, smoker ORDER BY time, smoker")
    
    sqlquery(conn,"UPDATE table1 SET smoker = 'Yes' WHERE total_bill = 14.78") # the UPDATE step - changes the smoker entry at ID 10 (not defined inherently in SQL database, so had to pull the $14.78 from entry 10 in spreadsheet) in tips.csv to 'Yes'
    
    sqlquery(conn,"DELETE FROM table1 WHERE total_bill < 10") # the DELETE step - removes all data entries with a total bill less than $10
    
except sqlite3.Error as e:
    print(f"Database connection error: {e}") # print error message if we cannot connect to the database
    
finally: # if everything in the "try" block is finished
    if conn: # and if the SQL connection still exists
        conn.close() # close the connection to the SQL database
        print("\nYour database connection has been closed.")

Connected to the SQL Database! Executing all queries...


Your database connection has been closed.


In [169]:
print("1. Retrieve the average tip percentage for each day of the week.") # question corresponding to SQL query 1
df1 = pd.DataFrame(sql1,columns=["Day","Mean Tip Percent"]) # create a table with appropriate columns
response1 = df1.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(response1) # display the table

1. Retrieve the average tip percentage for each day of the week.


Day,Mean Tip Percent
Fri,16.991303
Sat,15.315172
Sun,16.689729
Thur,16.127563


In [170]:
print("2. Find the minimum and maximum total bill amounts.") # question corresponding to SQL query 2
df2 = pd.DataFrame(sql2,columns=["Minimum Total Bill","Maximum Total Bill"]) # create a table with appropriate columns
response2 = df2.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(response2) # display the table

2. Find the minimum and maximum total bill amounts.


Minimum Total Bill,Maximum Total Bill
3.070000,50.810000


In [171]:
print("3. Count the number of parties for each size.") # question corresponding to SQL query 3
df3 = pd.DataFrame(sql3,columns=["Size","Number of Parties"]) # create a table with appropriate columns
response3 = df3.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(response3) # display the table

3. Count the number of parties for each size.


Size,Number of Parties
1,4
2,156
3,38
4,37
5,5
6,4


In [172]:
print("4. Retrieve the total bill and tip for parties of size 4 or more, where the tip percentage is greater than 15%.") # question corresponding to SQL query 4
df4 = pd.DataFrame(sql4,columns=["Size","Total Bill","Tip","Tip Percent"]) # create a table with appropriate columns
response4 = df4.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(response4) # display the table

4. Retrieve the total bill and tip for parties of size 4 or more, where the tip percentage is greater than 15%.


Size,Total Bill,Tip,Tip Percent
4,25.290000,4.710000,18.623962
4,18.430000,3.000000,16.277808
4,39.420000,7.580000,19.228818
4,30.400000,5.600000,18.421053
4,32.400000,6.000000,18.518519
4,25.560000,4.340000,16.979656
4,18.290000,3.760000,20.557682
4,29.930000,5.070000,16.939526
6,34.300000,6.700000,19.533528
6,27.050000,5.000000,18.484288


In [173]:
print("5. Retrieve the total bill, tip amount, and tip percentage for each combination of day and time, sorted by tip percentage in descending order.") # question corresponding to SQL query 5
df5 = pd.DataFrame(sql5,columns=["Day","Time","Tip","Total Bill","Tip Percent"]) # create a table with appropriate columns
response5 = df5.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(response5) # display the table

5. Retrieve the total bill, tip amount, and tip percentage for each combination of day and time, sorted by tip percentage in descending order.


Day,Time,Tip,Total Bill,Tip Percent
Fri,Dinner,4.300000,16.320000,26.348039
Fri,Dinner,2.500000,11.350000,22.026432
Fri,Dinner,3.000000,15.380000,19.505852
Fri,Dinner,1.000000,5.750000,17.391304
Fri,Dinner,3.500000,22.490000,15.562472
Fri,Dinner,4.000000,27.280000,14.662757
Fri,Dinner,3.250000,22.750000,14.285714
Fri,Dinner,3.000000,21.010000,14.278915
Fri,Dinner,1.500000,12.030000,12.468828
Fri,Dinner,1.500000,12.460000,12.038523


In [174]:
print("6. Find the average tip percentage for each combination of day, time, and smoker status.") # question corresponding to SQL query 6
df6 = pd.DataFrame(sql6,columns=["Day","Time","Smoker","Mean Tip Percent"]) # create a table with appropriate columns
response6 = df6.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(response6) # display the table

6. Find the average tip percentage for each combination of day, time, and smoker status.


Day,Time,Smoker,Mean Tip Percent
Fri,Dinner,No,13.962237
Fri,Dinner,Yes,16.534736
Fri,Lunch,No,18.773467
Fri,Lunch,Yes,18.893659
Sat,Dinner,No,15.804766
Sat,Dinner,Yes,14.790607
Sun,Dinner,No,16.011294
Sun,Dinner,Yes,18.725032
Thur,Dinner,No,15.974441
Thur,Lunch,No,16.031067


In [175]:
print("7. Retrieve the total bill, tip amount, and tip percentage for each sex, sorted by total bill in descending order, and limit the results to the top 5 records.") # question corresponding to SQL query 7
df7 = pd.DataFrame(sql7,columns=["Sex","Total Bill","Tip","Tip Percent"]) # create a table with appropriate columns
response7 = df7.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(response7) # display the table

7. Retrieve the total bill, tip amount, and tip percentage for each sex, sorted by total bill in descending order, and limit the results to the top 5 records.


Sex,Total Bill,Tip,Tip Percent
Male,50.810000,10.000000,19.681165
Male,48.330000,9.000000,18.621974
Male,48.270000,6.730000,13.942407
Male,48.170000,5.000000,10.379905
Male,45.350000,3.500000,7.717751


In [176]:
print("8. Find the maximum and minimum tip percentage for each day and time combination, along with the corresponding total bill and tip amount.") # question corresponding to SQL query 8
df8 = pd.DataFrame(sql8,columns=["Day","Time","Max or Min Tip Percent","Tip","Total Bill"]) # create a table with appropriate columns
response8 = df8.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(response8) # display the table

8. Find the maximum and minimum tip percentage for each day and time combination, along with the corresponding total bill and tip amount.


Day,Time,Max or Min Tip Percent,Tip,Total Bill
Fri,Dinner,10.355540,3.000000,28.970000
Fri,Dinner,26.348039,4.300000,16.320000
Fri,Lunch,11.773472,1.580000,13.420000
Fri,Lunch,25.931446,3.480000,13.420000
Sat,Dinner,3.563814,1.170000,32.830000
Sat,Dinner,32.573290,1.000000,3.070000
Sun,Dinner,5.944673,1.010000,16.990000
Sun,Dinner,71.034483,5.150000,7.250000
Thur,Dinner,15.974441,3.000000,18.780000
Thur,Lunch,7.296137,1.360000,18.640000


In [177]:
print("9. Retrieve the total bill, tip amount, and tip percentage for parties of size 4 or more, where the tip percentage is greater than 15%, and the total bill is between $50 and $100.\n") # question corresponding to SQL query 9
display(sql9) # display the output of the sqlquery function

9. Retrieve the total bill, tip amount, and tip percentage for parties of size 4 or more, where the tip percentage is greater than 15%, and the total bill is between $50 and $100.



'No results found.'

In [178]:
print("10. Find the average tip percentage for each combination of day, time, and smoker status, but only include combinations with more than 5 records.") # question corresponding to SQL query 10
df10 = pd.DataFrame(sql10,columns=["Day","Time","Smoker","Mean Tip Percent"]) # create a table with appropriate columns
response10 = df10.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(response10) # display the table

10. Find the average tip percentage for each combination of day, time, and smoker status, but only include combinations with more than 5 records.


Day,Time,Smoker,Mean Tip Percent
Fri,Dinner,Yes,16.534736
Fri,Lunch,Yes,18.893659
Sat,Dinner,No,15.804766
Sat,Dinner,Yes,14.790607
Sun,Dinner,No,16.011294
Sun,Dinner,Yes,18.725032
Thur,Lunch,No,16.031067
Thur,Lunch,Yes,16.386327


In [179]:
print("X1. Compute the total tip per day.") # question corresponding to extra SQL query 1
dfx1 = pd.DataFrame(xsql1,columns=["Day","Total Tip"]) # create a table with appropriate columns
responsex1 = dfx1.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(responsex1) # display the table

X1. Compute the total tip per day.


Day,Total Tip
Sat,260.400000
Sun,247.390000
Thur,171.830000
Fri,51.960000


In [180]:
print("X2. Compute the highest total bill for each party size.") # question corresponding to extra SQL query 2
dfx2 = pd.DataFrame(xsql2,columns=["Size","Highest Total Bill"]) # create a table with appropriate columns
responsex2 = dfx2.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(responsex2) # display the table

X2. Compute the highest total bill for each party size.


Size,Highest Total Bill
1,10.070000
2,40.550000
3,50.810000
4,48.330000
5,41.190000
6,48.170000


In [181]:
print("X3. Compute the average tip for each sex.") # question corresponding to extra SQL query 3
dfx3 = pd.DataFrame(xsql3,columns=["Sex","Mean Tip"]) # create a table with appropriate columns
responsex3 = dfx3.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(responsex3) # display the table

X3. Compute the average tip for each sex.


Sex,Mean Tip
Female,2.833448
Male,3.089618


In [182]:
print("X4. Compute the top 3 highest tips for each day.") # question corresponding to extra SQL query 4
dfx4 = pd.DataFrame(xsql4,columns=["Day","3 Highest Tips Per Day"]) # create a table with appropriate columns
responsex4 = dfx4.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(responsex4) # display the table

X4. Compute the top 3 highest tips for each day.


Day,3 Highest Tips Per Day
Fri,4.730000
Fri,4.300000
Fri,4.000000
Sat,10.000000
Sat,9.000000
Sat,7.580000
Sun,6.500000
Sun,6.000000
Sun,5.650000
Thur,6.700000


In [183]:
print("X5. Compute the number of smokers vs. number of non-smokers by time.") # question corresponding to extra SQL query 1
dfx5 = pd.DataFrame(xsql5,columns=["Time","Smoker","Count"]) # create a table with appropriate columns
responsex5 = dfx5.style.hide_index().set_properties(**{'background-color': 'lightblue','border': '2px solid black'}) # format table to remove row labels and add a colored background
display(responsex5) # display the table

X5. Compute the number of smokers vs. number of non-smokers by time.


Time,Smoker,Count
Dinner,No,106
Dinner,Yes,70
Lunch,No,45
Lunch,Yes,23
